### 1-1. 2이하 길이의 이름 갯수 확인

In [1]:
import csv
import json
import pandas as pd 

nameState = pd.read_csv('C:/Users/SSAFY/S07P22A108/DataSet/Processing Data/name_state.csv')

under2 = nameState[nameState["name"].apply(lambda x:len(str(x))==2)]
under2

,state,gender,year,name,count
1092,AK,F,1947,Jo,5
1243,AK,F,1949,Jo,6
1309,AK,F,1950,Jo,10
1517,AK,F,1952,Jo,10
1674,AK,F,1953,Jo,9
...,...,...,...,...,...
6308987,WY,M,2006,Ty,10
6309196,WY,M,2007,Ty,7
6309711,WY,M,2010,Ty,8
6309860,WY,M,2011,Ty,11


### 1-2. 2이상 길이의 이름만 남기기

In [2]:
removeUnder2 = nameState[nameState["name"].apply(lambda x:len(str(x))>2)]
removeUnder2

,state,gender,year,name,count
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7
...,...,...,...,...,...
6311499,WY,M,2021,Santiago,5
6311500,WY,M,2021,Spencer,5
6311501,WY,M,2021,Tate,5
6311502,WY,M,2021,Timothy,5


### 2-1. 1940년 미만 데이터 갯수 확인 (from removeUnder2)

In [3]:
under1940 = removeUnder2[removeUnder2['year']<1940]
under1940

,state,gender,year,name,count
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7
...,...,...,...,...,...
6299611,WY,M,1939,Roland,5
6299612,WY,M,1939,Ronnie,5
6299613,WY,M,1939,Tommy,5
6299614,WY,M,1939,Tony,5


### 2-2. 1940년 이상 데이터만 남기기

In [4]:
over1940 = removeUnder2[removeUnder2['year']>=1940]
over1940

,state,gender,year,name,count
687,AK,F,1940,Mary,43
688,AK,F,1940,Barbara,15
689,AK,F,1940,Patricia,15
690,AK,F,1940,Alice,14
691,AK,F,1940,Annie,12
...,...,...,...,...,...
6311499,WY,M,2021,Santiago,5
6311500,WY,M,2021,Spencer,5
6311501,WY,M,2021,Tate,5
6311502,WY,M,2021,Timothy,5


### 3. Behind the Names랑 데이터 비교

- Behind에 있는 이름의 갯수(중복 제거)

In [5]:
ssaNames = over1940
behindNames = pd.read_json("C:/Users/SSAFY/S07P22A108/DataSet/Processing Data/mvpNameSet_merge_normal_abnormal.json")

print('데이터 열의 개수 : '+str(len(ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())])))

print('이름의 개수 : '+str(len(ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())].drop_duplicates('name'))))

데이터 열의 개수 : 3861500
이름의 개수 : 3626


- Behind the Names에 없는 이름의 갯수

In [6]:
ssaNames = over1940
behindNames = pd.read_json("C:/Users/SSAFY/S07P22A108/DataSet/Processing Data/mvpNameSet_merge_normal_abnormal.json")

print('데이터 열의 개수 : '+str(len(ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())==False])))

print('이름의 개수 : '+str(len(ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())==False].drop_duplicates('name'))))


데이터 열의 개수 : 1428193
이름의 개수 : 26476


### 4. Behind에 있는 이름 데이터 수집 (빈도수, 주, 년 수집)

In [7]:

ssaNames = ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())]

    

In [8]:

with open('C:/Users/SSAFY/S07P22A108/DataSet/Processing Data/mvpNameSet_Behind_Fin_with_default_state.json', 'r') as f:
    tempBehindNames = json.load(f)
        
behindNamesDict = {}
for name in tempBehindNames:
    behindNamesDict[name['name'].lower()] = name

ssaNames = ssaNames[ssaNames['name'].str.lower().isin(behindNames['name'].str.lower())] 

index = 0
for ssaName in ssaNames.itertuples():
    if ssaName.gender == 'F' and behindNamesDict[ssaName.name.lower()]['female']!={}:
        behindNamesDict[ssaName.name.lower()]['female']['state'][ssaName.state][ssaName.year-1940] += ssaName.count
    if ssaName.gender == 'M' and behindNamesDict[ssaName.name.lower()]['male']!={}:
        behindNamesDict[ssaName.name.lower()]['male']['state'][ssaName.state][ssaName.year-1940] += ssaName.count

nameSet = []
for k,v in behindNamesDict.items():
    nameSet.append(v)
    
fileName = "C:/Users/SSAFY/S07P22A108/DataSet/Processing Data/mvpNameSet_Behind_Fin_with_count_state.json"
file = open(fileName, "w")
json.dump(nameSet, file)
file.close()
print("fin")

fin
